# Funções
Arquivo de funções de machine Learning para prever boas apostas de futebol para a Bet365 na temporada 2021/2022 da Premier League

In [32]:
import pandas as pd
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import re
import glob

In [33]:
# Lista de caminhos para os arquivos CSV de cada temporada
paths = glob.glob('../data/seasons/*.csv')

# Lista para armazenar os datasets de cada temporada
datasets = []

# Carregar os datasets de cada temporada
for path in paths:
    # Extrair o nome da temporada do caminho do arquivo
    temporada = path.split('/')[-1].split('.')[0]
    
    # Carregar o dataset da temporada e adicionar a coluna 'Temporada'
    dataset = pd.read_csv(path)
    dataset['Temporada'] = temporada
    
    # Adicionar o dataset à lista
    datasets.append(dataset)

# Concatenar os datasets em um único dataframe
data = pd.concat(datasets, ignore_index=True)

# Substituir valores ausentes por valores nulos (NaN)
data.replace('?', pd.NA, inplace=True)

# Exibir o dataframe resultante
print(data.head())


  Div      Date  HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  ...  \
0  E0  19/08/00  Charlton       Man City     4     0   H     2     0   H  ...   
1  E0  19/08/00   Chelsea       West Ham     4     2   H     1     0   H  ...   
2  E0  19/08/00  Coventry  Middlesbrough     1     3   A     1     1   D  ...   
3  E0  19/08/00     Derby    Southampton     2     2   D     1     2   A  ...   
4  E0  19/08/00     Leeds        Everton     2     0   H     2     0   H  ...   

   AvgC<2.5 AHCh  B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  MaxCAHA  AvgCAHH  \
0       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   
1       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   
2       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   
3       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   
4       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   

   AvgCAHA  
0      NaN  


In [34]:
# Filtrar os resultados antes de 31/05/2022
train_data = data[data['Date'] < '2022-05-31']

# Selecionar as colunas relevantes para treinamento
train_data = train_data[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "FTR"]]


# Converta as variáveis categóricas em variáveis numéricas usando one-hot encoding
train_data_encoded = pd.get_dummies(train_data, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_train = train_data_encoded.drop("FTR", axis=1)
y_train = train_data_encoded["FTR"]

# Filtrar os resultados após 31/05/2022
test_data = data[data['Date'] > '2022-05-31']

# Selecionar as colunas relevantes para teste
test_data = test_data[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "FTR"]]

# Converta as variáveis categóricas em variáveis numéricas usando one-hot encoding
test_data_encoded = pd.get_dummies(test_data, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_test = test_data_encoded.drop("FTR", axis=1)
y_test = test_data_encoded["FTR"]

# Crie e ajuste o modelo de regressão logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Crie e ajuste o modelo de árvore de decisão
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

# Crie e ajuste o modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Crie e ajuste o modelo SVM
svm = SVC()
svm.fit(X_train, y_train)

# Crie e ajuste o modelo Naive Bayes
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)

# Avalie a acurácia dos modelos
accuracy_logreg = logreg.score(X_test, y_test)
accuracy_decision_tree = decision_tree.score(X_test, y_test)
accuracy_random_forest = random_forest.score(X_test, y_test)
accuracy_svm = svm.score(X_test, y_test)
accuracy_naive_bayes = naive_bayes.score(X_test, y_test)

# Imprima as acurácias
print("Acurácia (Regressão Logística):", accuracy_logreg)
print("Acurácia (Árvore de Decisão):", accuracy_decision_tree)
print("Acurácia (Random Forest):", accuracy_random_forest)
print("Acurácia (SVM):", accuracy_svm)
print("Acurácia (Naive Bayes):", accuracy_naive_bayes)

print(train_data_encoded)

c:\Users\guifl\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Acurácia (Regressão Logística): 1.0
Acurácia (Árvore de Decisão): 0.9990205680705191
Acurácia (Random Forest): 0.9924910218739798
Acurácia (SVM): 0.9986940907606922
Acurácia (Naive Bayes): 0.5213842637936663
      FTHG  FTAG  HS  AS  HST  AST FTR  HomeTeam_Aston Villa  \
0        4     0  17   8   14    4   H                     0   
1        4     2  17  12   10    5   H                     0   
2        1     3   6  16    3    9   A                     0   
3        2     2   6  13    4    6   D                     0   
4        2     0  17  12    8    6   H                     0   
...    ...   ...  ..  ..  ...  ...  ..                   ...   
8385     2     0  16  11    4    2   H                     0   
8386     1     2   4  24    2    5   A                     0   
8387     3     2  12   7    6    5   H                     0   
8388     1     1  22  10    9    5   D                     1   
8389     1     1  20   2    7    1   D                     0   

      HomeTeam_Birmingh

In [35]:
def obter_estatisticas_confronto(home_team, away_team):
    # Filtrar os jogos anteriores entre os times
    confrontos = data[((data['HomeTeam'] == home_team) & (data['AwayTeam'] == away_team)) |
                      ((data['HomeTeam'] == away_team) & (data['AwayTeam'] == home_team))]

    # Calcular as estatísticas dos confrontos
    total_jogos = confrontos.shape[0]
    vitorias_home = confrontos[confrontos['HomeTeam'] == home_team]['FTR'].eq('H').sum()
    vitorias_away = confrontos[confrontos['AwayTeam'] == home_team]['FTR'].eq('A').sum()
    empates = confrontos[confrontos['FTR'] == 'D'].shape[0]

    # Criar DataFrame com as estatísticas
    estatisticas = pd.DataFrame({'TotalJogos': [total_jogos],
                                 'VitoriasHome': [vitorias_home],
                                 'VitoriasAway': [vitorias_away],
                                 'Empates': [empates]})

    return estatisticas


In [36]:
def obter_medias_times(home_team, away_team):
    # Filtrar os jogos do time
    jogos = data[(data['HomeTeam'] == home_team) & (data['AwayTeam'] == away_team)]

    # Calcular as médias de gols, chutes e chutes no alvo
    home_media_gols = jogos['FTHG'].mean()
    home_media_chutes = jogos['HS'].mean()
    home_media_chutes_alvo = jogos['HST'].mean()
    away_media_gols = jogos['FTAG'].mean()
    away_media_chutes = jogos['AS'].mean()
    away_media_chutes_alvo = jogos['AST'].mean()

    # Criar DataFrame com as médias
    medias = pd.DataFrame({'HomeTeam': [home_team],
                            'AwayTeam': [away_team],
                            'FTHG': [home_media_gols],
                            'HS': [home_media_chutes],
                            'HST': [home_media_chutes_alvo],
                            'FTAG': [away_media_gols],
                            'AS': [away_media_chutes],
                            'AST': [away_media_chutes_alvo]})

    # Adicionar colunas de codificação dos times
    for col in X_train.columns:
        if col.startswith("HomeTeam_") and col != f"HomeTeam_{home_team}":
            medias[col] = 0
        elif col.startswith("AwayTeam_") and col != f"AwayTeam_{away_team}":
            medias[col] = 0
        elif col.startswith("HomeTeam_") and col == f"HomeTeam_{home_team}":
            medias[col] = 1
        elif col.startswith("AwayTeam_") and col == f"AwayTeam_{away_team}":
            medias[col] = 1
    
    # Reordenar as colunas
    medias = medias[X_train.columns]
    

    # Adicionar colunas ausentes
    missing_cols = set(X_train.columns) - set(medias.columns)
    for col in missing_cols:
        medias[col] = 0

    return medias


In [37]:
db = pathlib.Path.cwd() / "../data/seasons/2022-2023.csv"
lastseason = pd.read_csv(db)

# Função para fazer uma sugestão de aposta com base no modelo treinado
def aposta_resultado(jogo, modelo):
    jogo_encoded = obter_medias_times(jogo['HomeTeam'], jogo['AwayTeam'])

    # Fazer a previsão com base no modelo treinado
    predicao = modelo.predict(jogo_encoded)
    probabilidade = modelo.predict_proba(jogo_encoded)

    # Filtrar o jogo específico
    jogo_especifico = lastseason[(lastseason['HomeTeam'] == jogo['HomeTeam']) & (lastseason['AwayTeam'] == jogo['AwayTeam'])]

    # Obter as odds específicas do jogo
    odd_home = jogo_especifico['B365H'].values[0]
    odd_draw = jogo_especifico['B365D'].values[0]
    odd_away = jogo_especifico['B365A'].values[0]

    # Criar o dicionário com a sugestão de aposta
    sugestao_aposta = {
        'HomeTeam': jogo['HomeTeam'],
        'AwayTeam': jogo['AwayTeam'],
        'Predicao': predicao[0],
        'OddHome': odd_home,
        'OddDraw': odd_draw,
        'OddAway': odd_away,
    }

    # Se a previsão for de vitória do time da casa
    if predicao[0] == 'H':
        sugestao = "Sugestão de aposta: Vitória do time da casa com odd de {}".format(odd_home)
    # Se a previsão for de empate
    elif predicao[0] == 'D':
        sugestao = "Sugestão de aposta: Empate com odd de {}".format(odd_draw)
    # Se a previsão for de vitória do time visitante
    else:
        sugestao = "Sugestão de aposta: Vitória do time visitante com odd de {}".format(odd_away)

    return sugestao

# Carregar o modelo treinado
modelo = decision_tree

# Exemplo de jogo específico
jogo = {
    "HomeTeam": 'Man United',
    "AwayTeam": 'Arsenal',
}

# Fazer a sugestão de aposta com base no modelo treinado
sugestao_aposta = aposta_resultado(jogo, modelo)

print(sugestao_aposta)


Sugestão de aposta: Vitória do time da casa com odd de 1.95


# Funções para > 2.5 gols e < 2.5 gols (over e under) 

In [38]:
# Filtrar os resultados antes de 31/05/2022
gols_train = data[data['Date'] < '2022-05-31']

# Selecionar as colunas relevantes para treinamento
gols_train = gols_train[["HomeTeam", "AwayTeam", "FTHG", "FTAG"]]

# Transformar a variável de saída em uma variável binária indicando se teve mais ou menos de 2.5 gols
gols_train['Resultado'] = gols_train['FTHG'] + gols_train['FTAG'] > 2.5
gols_train['Resultado'] = gols_train['Resultado'].astype(int)

# Converter as variáveis categóricas em variáveis numéricas usando one-hot encoding
gols_train_encoded = pd.get_dummies(gols_train, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_train_gols = gols_train_encoded.drop("Resultado", axis=1)
y_train_gols = gols_train_encoded["Resultado"]

# Filtrar os resultados a partir de 31/05/2022
test_gols = data[data['Date'] >= '2022-05-31']

# Selecionar as colunas relevantes para teste
test_gols = test_gols[["HomeTeam", "AwayTeam", "FTHG", "FTAG"]]
test_gols['Resultado'] = test_gols['FTHG'] + test_gols['FTAG'] > 2.5
test_gols['Resultado'] = test_gols['Resultado'].astype(int)

# Converter as variáveis categóricas em variáveis numéricas usando one-hot encoding
test_gols_encoded = pd.get_dummies(test_gols, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_test_gols = test_gols_encoded.drop("Resultado", axis=1)
y_test_gols = test_gols_encoded["Resultado"]

# Crie e ajuste o modelo de regressão logística
logreg_gols = LogisticRegression()
logreg_gols.fit(X_train_gols, y_train_gols)

# Crie e ajuste o modelo de árvore de decisão
decision_tree_gols = DecisionTreeClassifier(random_state=42)
decision_tree_gols.fit(X_train_gols, y_train_gols)

# Crie e ajuste o modelo de Random Forest
random_forest_gols = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_gols.fit(X_train_gols, y_train_gols)

# Crie e ajuste o modelo SVM_gols
svm_gols = SVC()
svm_gols.fit(X_train_gols, y_train_gols)

# Crie e ajuste o modelo Naive Bayes
naive_bayes_gols = GaussianNB()
naive_bayes_gols.fit(X_train_gols, y_train_gols)

# Avalie a acurácia dos modelos
accuracy_logreg_gols = logreg_gols.score(X_test_gols, y_test_gols)
accuracy_decision_tree = decision_tree_gols.score(X_test_gols, y_test_gols)
accuracy_random_forest_gols = random_forest_gols.score(X_test_gols, y_test_gols)
accuracy_svm_gols = svm_gols.score(X_test_gols, y_test_gols)
accuracy_naive_bayes_gols = naive_bayes_gols.score(X_test_gols, y_test_gols)

# Imprima as acurácias
print("Acurácia (Regressão Logística):", accuracy_logreg_gols)
print("Acurácia (Árvore de Decisão):", accuracy_decision_tree)
print("Acurácia (Random Forest):", accuracy_random_forest_gols)
print("Acurácia (SVM_gols):", accuracy_svm_gols)
print("Acurácia (Naive Bayes):", accuracy_naive_bayes_gols)

print(gols_train_encoded)

Acurácia (Regressão Logística): 1.0
Acurácia (Árvore de Decisão): 1.0
Acurácia (Random Forest): 1.0
Acurácia (SVM_gols): 1.0
Acurácia (Naive Bayes): 0.6885406464250735
      FTHG  FTAG  Resultado  HomeTeam_Aston Villa  HomeTeam_Birmingham  \
0        4     0          1                     0                    0   
1        4     2          1                     0                    0   
2        1     3          1                     0                    0   
3        2     2          1                     0                    0   
4        2     0          0                     0                    0   
...    ...   ...        ...                   ...                  ...   
8385     2     0          0                     0                    0   
8386     1     2          1                     0                    0   
8387     3     2          1                     0                    0   
8388     1     1          0                     1                    0   
8389     1     1  

In [51]:
# Função para fazer uma sugestão de aposta com base no modelo treinado
def fazer_sugestao_aposta(jogo, modelo):
    jogo_encoded = obter_medias_times(jogo['HomeTeam'], jogo['AwayTeam'])
    # cortar as colunas AS, AST, HS e HST
    jogo_encoded = jogo_encoded.drop(['AS', 'AST', 'HS', 'HST'], axis=1)

    # Fazer a previsão com base no modelo treinado
    predicao = modelo.predict(jogo_encoded)
    probabilidade = modelo.predict_proba(jogo_encoded)

    # Filtrar o jogo específico
    jogo_especifico = lastseason[(lastseason['HomeTeam'] == jogo['HomeTeam']) & (lastseason['AwayTeam'] == jogo['AwayTeam'])]

    # Obter as odds específicas do jogo
    odd_maior = jogo_especifico['B365>2.5'].values[0]
    odd_menor = jogo_especifico['B365<2.5'].values[0]

    # Criar o dicionário com a sugestão de aposta
    sugestao_aposta = {
        'HomeTeam': jogo['HomeTeam'],
        'AwayTeam': jogo['AwayTeam'],
        'Predicao': predicao[0],
        'OddMaior': odd_maior,
        'OddMenor': odd_menor,
    }

    # Verificar se o jogo teve mais de 2.5 gols ou não
    if predicao[0]:
        sugestao = "Sugestão de aposta: Mais de 2.5 gols com odd de {}".format(odd_maior)
    else:
        sugestao = "Sugestão de aposta: Menos de 2.5 gols com odd de {}".format(odd_menor)

    return sugestao

# Carregar o modelo treinado
modelo = logreg_gols

# Exemplo de jogo específico
jogo = {
    "HomeTeam": 'Man United',
    "AwayTeam": 'Arsenal',
}

# Fazer a sugestão de aposta com base no modelo treinado
sugestao_aposta = fazer_sugestao_aposta(jogo, modelo)

print(sugestao_aposta)

Sugestão de aposta: Mais de 2.5 gols com odd de 1.8


# Handicap Asiático: Juntando as funções

In [43]:
# Filtrar os resultados antes de 31/05/2022
handicap_train = data[data['Date'] < '2022-05-31']

# Selecionar as colunas relevantes para treinamento
handicap_train = handicap_train[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST"]]

# Criar a coluna de handicap asiático
handicap_train['Handicap'] = np.where(handicap_train['FTHG'] > handicap_train['FTAG'], 1, -1)

# Converter as variáveis categóricas em variáveis numéricas usando one-hot encoding
handicap_train_encoded = pd.get_dummies(handicap_train, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_train_handicap = handicap_train_encoded.drop("Handicap", axis=1)
y_train_handicap = handicap_train_encoded["Handicap"]

# Filtrar os resultados a partir de 31/05/2022
test_handicap = data[data['Date'] >= '2022-05-31']

# Selecionar as colunas relevantes para teste
test_handicap = test_handicap[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST"]]

# Criar a coluna de handicap asiático
test_handicap['Handicap'] = np.where(test_handicap['FTHG'] > test_handicap['FTAG'], 1, -1)

# Converter as variáveis categóricas em variáveis numéricas usando one-hot encoding
test_handicap_encoded = pd.get_dummies(test_handicap, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_test_handicap = test_handicap_encoded.drop("Handicap", axis=1)
y_test_handicap = test_handicap_encoded["Handicap"]

# Crie e ajuste o modelo de regressão logística
logreg_handicap = LogisticRegression()
logreg_handicap.fit(X_train_handicap, y_train_handicap)

# Crie e ajuste o modelo de árvore de decisão
decision_tree_handicap = DecisionTreeClassifier(random_state=42)
decision_tree_handicap.fit(X_train_handicap, y_train_handicap)

# Crie e ajuste o modelo de Random Forest
random_forest_handicap = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_handicap.fit(X_train_handicap, y_train_handicap)

# Crie e ajuste o modelo SVM_handicap
svm_handicap = SVC()
svm_handicap.fit(X_train_handicap, y_train_handicap)

# Crie e ajuste o modelo Naive Bayes
naive_bayes_handicap = GaussianNB()
naive_bayes_handicap.fit(X_train_handicap, y_train_handicap)

# Avalie a acurácia dos modelos
accuracy_logreg_gols = logreg_handicap.score(X_test_handicap, y_test_handicap)
accuracy_decision_tree = decision_tree_handicap.score(X_test_handicap, y_test_handicap)
accuracy_random_forest_handicap = random_forest_handicap.score(X_test_handicap, y_test_handicap)
accuracy_svm_handicap = svm_handicap.score(X_test_handicap, y_test_handicap)
accuracy_naive_bayes_handicap = naive_bayes_handicap.score(X_test_handicap, y_test_handicap)

# Imprima as acurácias
print("Acurácia (Regressão Logística):", accuracy_logreg_gols)
print("Acurácia (Árvore de Decisão):", accuracy_decision_tree)
print("Acurácia (Random Forest):", accuracy_random_forest_handicap)
print("Acurácia (SVM_handicap):", accuracy_svm_handicap)
print("Acurácia (Naive Bayes):", accuracy_naive_bayes_handicap)


c:\Users\guifl\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Acurácia (Regressão Logística): 1.0
Acurácia (Árvore de Decisão): 0.9986940907606922
Acurácia (Random Forest): 0.9986940907606922
Acurácia (SVM_handicap): 1.0
Acurácia (Naive Bayes): 0.6989879203395364


In [50]:
def fazer_sugestao_aposta(jogo, modelo_vitoria, modelo_handicap):
    jogo_encoded = obter_medias_times(jogo['HomeTeam'], jogo['AwayTeam'])

    # Fazer a previsão com base no modelo treinado
    vencedor = modelo_vitoria.predict(jogo_encoded)
    probabilidade = modelo_vitoria.predict_proba(jogo_encoded)

    # Fazer a previsão com base no modelo treinado
    handicap = modelo_handicap.predict(jogo_encoded)

    # Filtrar o jogo específico
    jogo_especifico = lastseason[(lastseason['HomeTeam'] == jogo['HomeTeam']) & (lastseason['AwayTeam'] == jogo['AwayTeam'])]

    # Obter a diferença de gols prevista
    diferenca_gols = int(handicap[0])


    # Obter a odd do handicap asiático
    odd_handicap = jogo_especifico['B365AHH'].values[0]

    # Verificar o vencedor e o handicap a ser apostado
    if vencedor[0] == 'H':
        vencedor = jogo['HomeTeam']
        handicap = diferenca_gols
        sugestao = f"Sugestão de aposta: Vitória do time da casa ({vencedor}) com handicap asiático de {handicap} com odd de {odd_handicap}"

    elif vencedor[0] == 'A':
        vencedor = jogo['AwayTeam']
        handicap = -diferenca_gols
        sugestao = f"Sugestão de aposta: Vitória do time visitante ({vencedor}) com handicap asiático de {handicap} com odd de {odd_handicap}"
    else:
        sugestao = f"Sugestão de aposta: Empate"
    

    return sugestao

# Exemplo de jogo específico
jogo = {
    "HomeTeam": 'Man City',
    "AwayTeam": 'Chelsea',
}

# Fazer a sugestão de aposta com base no modelo treinado
sugestao_aposta = fazer_sugestao_aposta(jogo, decision_tree, decision_tree_handicap)

print(sugestao_aposta)


Sugestão de aposta: Empate
